# Оабораторная работа №2 по вычислительной математике

## Прменение прямых и итерационных методов для решения СЛАУ

### Выполнил Филиппенко Павел -- студент группы Б01-009

In [549]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import math

##### Зададим нормы векторов

$$||x||_1 = max_i |x_i|$$

$$||x||_2 = \sum |x_i|$$

$$||x||_2 = (x, x)$$

In [550]:
def fst_vec_norm(x: np.ndarray):
    return max(abs(x))

def scd_vec_norm(x: np.ndarray):
    return sum(abs(x))

def trd_vec_norm(x: np.ndarray):
    return math.sqrt(np.dot(x, x))

##### Зададим нормы матриц

(по строкам)
$$||A||_1 = \max \limits_i \sum_j |a_{ij}|$$

(по столбцам)
$$||A||_2 = \max \limits_j \sum_i |a_{ij}|$$

$$||A||_3 = \sqrt{\max \limits_i \lambda_i(A^* A)}$$

Поскольку в данной работе мы рассмотриваем матрицы действительного пространства, $A^* = A^T$

In [551]:
def fst_m_norm(A: np.ndarray):
    assert(A.shape[0] == A.shape[1])
    return max([sum(abs(A[i])) for i in range(A.shape[0])])

def scd_m_norm(A: np.ndarray):
    assert(A.shape[0] == A.shape[1])
    return max([sum(abs(A.T[i])) for i in range(A.T.shape[0])])

# поскольку работаем в R, эрмитово сопряжение эквивалентвно транспонированию
def trd_m_norm(A: np.ndarray):
    B = np.dot(A.T, A)
    num, _ =  np.linalg.eigh(B)
    print(num)
    return math.sqrt(max(num))

##### Класс Slae представляет систему линейных уравнений.

_Поля_:
- A -- матрица системы
- f -- столбец решений

_Методы_:
- dimention -- декоратор, возвращающий порядок системы
- check_symmetric -- проверка матрицы на симметричность
- Gauss_mthd   -- решение системы линейных уравнений методом Гауса
- LU_mthd      -- решение системы линейных уравнений методом LU-разложения
- Holecky_mthd -- решение системы линейных уравнений методом Холецкого
- Zaydel_mthd  -- решение системы линейных уравнений методом Зейделя

In [552]:
# порядок округдения коэффициентов (нужен исключительно для вывода)
# при вычислениях коэффициенты НЕ ОКРУГЛЯЮТСЯ
round_n = 3

class Slae:
    def __init__(self, matrix: np.ndarray, values: np.ndarray):

        # проверяем, что матрица квадратная и вектор значений имеет соответсвующую размерность
        assert(matrix.shape[0] == matrix.shape[1])
        assert(matrix.shape[0] == values.shape[0])
        
        self.A = matrix
        self.f = values

    #================================================Приватные методы и декараторы================================================#

    @property
    def dimention(self):
        return self.A.shape[0]

    # метод проверяет матрицу на симметричность
    def CheckSymmetric(self, tol=1e-16):
        return not False in (np.abs(self.A-self.A.T) < tol)

    # проверка главных угловых миноров для LU разложения
    def IsLU_compatible(self):
        N = self.dimention
        for i in range(1, N+1):
            M = self.A[:i, :i]

            if np.linalg.det(M) == 0:
                return False

        return True

    # проверка матрицы на положительную определенность
    def SylvesterCriterion(self):
        N = self.dimention
        for i in range(1, N+1):
            M = self.A[:i, :i]

            if np.linalg.det(M) < 0:
                return False

        return True

    def LU_decomposition(self):
        a = self.A
        """Decompose matrix of coefficients to L and U matrices.
        L and U triangular matrices will be represented in a single nxn matrix.
        :param a: numpy matrix of coefficients
        :return: numpy LU matrix
        """
        # create emtpy LU-matrix
        lu_matrix = np.matrix(np.zeros([a.shape[0], a.shape[1]]))
        n = a.shape[0]

        for k in range(n):
            # calculate all residual k-row elements
            for j in range(k, n):
                lu_matrix[k, j] = a[k, j] - lu_matrix[k, :k] * lu_matrix[:k, j]
            # calculate all residual k-column elemetns
            for i in range(k + 1, n):
                lu_matrix[i, k] = (a[i, k] - lu_matrix[i, : k] * lu_matrix[: k, k]) / lu_matrix[k, k]

        """Get triangular L matrix from a single LU-matrix
        :param m: numpy LU-matrix
        :return: numpy triangular L matrix
        """
        L = lu_matrix.copy()
        for i in range(L.shape[0]):
                L[i, i] = 1
                L[i, i+1 :] = 0

        """Get triangular U matrix from a single LU-matrix
        :param m: numpy LU-matrix
        :return: numpy triangular U matrix
        """
        U = lu_matrix.copy()
        for i in range(1, U.shape[0]):
            U[i, :i] = 0
        
        return L, U
    #==================================================Численные методы==================================================#

    def Gauss_mthd(self):
        # размерность матрицы
        n = self.dimention

        # прямой ход метода Гауса
        for k in range(n):
            for m in range(k+1, n):

                alpha = self.A[m][k] / self.A[k][k]

                self.f[m] = self.f[m] - self.f[k] * alpha 
                for i in range(k, n):
                    self.A[m][i] = self.A[m][i] - self.A[k][i] * alpha

        # обратный ход
        solution = np.full((n, ), 0.0)
        
        # поскольку индексы в python начинаются с 0 и заканчиваются n-1, последнее уравнение имеет индекс n-1
        solution[n-1] = self.f[n-1] / self.A[n-1][n-1]

        # предпоследнее уравнение имеет интекс n-2
        # поскольку функция range возвращает полуоткрытый интервал, вторым параметром ей передаеся -1, а не 0
        for k in range(n-2, 0-1, -1):
            solution[k] = 1 / self.A[k][k] * (self.f[k] - np.dot(self.A[k], solution))

        return solution


    def LU_mthd(self):
        A = self.A
        N = self.dimention

        L, U = self.LU_decomposition()

        solution_level1 = np.full((N, ), 0.0)
        solution_level2 = np.full((N, ), 0.0)

        solution_level1[0] = self.f[0] / L[0, 0]

        for i in range(1, N):
            solution_level1[i] = 1 / L[i, i] * (self.f[i] - np.dot(L[i], solution_level1))

        solution_level2[N-1] = solution_level1[N-1] / U[N-1, N-1]

        
        for k in range(N-2, 0-1, -1):
            solution_level2[k] = 1 / U[k, k] * (solution_level1[k] - np.dot(U[k], solution_level2))

        return solution_level2


    # для метода Холецкого необходимо, чтобы матрица была положительно определена и симметрична
    def Cholesky_mthd(self):
        A = self.A
        N = self.dimention
        L = np.zeros([A.shape[0], A.shape[1]])

        for j in range(0, A.shape[0]):
            LSum = 0.0
            for k in range(0, j):
                LSum += L[j, k] * L[j, k]

            L[j, j] = np.sqrt(A[j, j] - LSum)

            for i in range(j + 1, A.shape[1]):
                LSum = 0.0
                for k in range(0, j):
                    LSum += L[i, k] * L[j, k]
                L[i][j] = (1.0 / L[j, j] * (A[i, j] - LSum))
        
        solution_level1 = np.full((N, ), 0.0)
        solution_level2 = np.full((N, ), 0.0)

        solution_level1[0] = self.f[0] / L[0, 0]

        U = L.T

        for i in range(1, N):
            solution_level1[i] = 1 / L[i, i] * (self.f[i] - np.dot(L[i], solution_level1))

        solution_level2[N-1] = solution_level1[N-1] / U[N-1, N-1]

        
        for k in range(N-2, 0-1, -1):
            solution_level2[k] = 1 / U[k, k] * (solution_level1[k] - np.dot(U[k], solution_level2))

        return solution_level2
        
        


    def UpperRelaxation(self, w = 1.5):        
        eps = 1e-6
        N = self.dimention

        D = np.eye(N) * np.diag(self.A)
        U = np.triu(self.A) - D
        L = np.tril(self.A) - D

        B = np.dot(np.linalg.inv(L*w + D), D*(w - 1) + U*w)
        F = np.linalg.inv(L*w + D)

        iteration_num = 50

        solution_prev = np.full((N, ), 0.0)
        solution_cur = np.full((N, ), 0.0)

        while(trd_vec_norm(self.f - np.dot(self.A, solution_cur)) > eps):
            solution_prev = solution_cur
            solution_cur = - np.dot(B, solution_prev) + np.dot(F*w, self.f)
            

        return solution_cur

    def Seidel_mthd(self):
        res = self.UpperRelaxation(w=1)
        return res

    #==================================================Другие функции класса==================================================#

    ## overloading output
    def __str__(self):
        n = self.dimention

        res = ''
        for i in range(n):
            string = ''
            for j in range(n):
                string = string + str(round(self.A[i][j], round_n)) + ' x{}'.format(j + 1)
                # string = string + str(self.A[i][j]) + ' x{}'.format(j + 1)
                if j != n - 1:
                    string = string + ' + '
                else:
                    string = string + ' = ' + str(round(self.f[i], round_n))
                    # string = string + ' = ' + str(self.f[i])
            string = string + '\n'
            res = res + string

        return res

##### Зададим систему уравнений через матрицу и столбей решений.

In [553]:
# N = 12
# A = np.eye(N)
# f = np.full((N, ), 1.0)


# for i in range(N):
#     for j in range(N):
#         if i == j:
#             A[i][j] = 1
#         else:
#             A[i][j] = 1 / ((i+1)**2 + (j+2))
#     f[i] = 1 / (i + 1)

# eq = Slae(A, f)

### TODO List

- В каждом методе написать копирование матрицы, иначе, они передаются по ссылке
- Написать ограничения на решение методами Холецкого и Зейделя
- Рефакторинг имен переменных в методах и комметарии
- Написать расчет невязок
- Написать вычисление количества шагов итераций
- Написать степенной метод нахождения наибольшего собственного числа и провести проверку на разных алгоритмах и начальных векторах. Написать вывод по этому поводу.
- Оформление отчета.